In [15]:
import json
import torch
from sentence_transformers import util

In [16]:
small = "stexpanded"
big = "memoryalpha"
embeddings = "dogtag_bgelarge"
top = 1000

In [17]:
mappings_file_small = "./_input/mappings/" + small + ".json"
mappings_file_big = "./_input/mappings/" + big + ".json"

node_embeddings_small_file = "./_input/node_embeddings/" + embeddings + "/" + small + ".json"
node_embeddings_big_file = "./_input/node_embeddings/" + embeddings + "/" + big + ".json"

exact_match_file = "./_input/exact_match/" + small + "-" + big + ".json"
gold_pairs_file = "./_input/gold_pairs/" + small + "-" + big + ".txt"

In [18]:
with open(mappings_file_small) as file:
    mappings_small = json.load(file)
    mappings_small = {str(v): k for k, v in mappings_small.items()}
    mappings_small_reversed = {v: k for k, v in mappings_small.items()}

with open(mappings_file_big) as file:
    mappings_big = json.load(file)
    mappings_big = {str(v): k for k, v in mappings_big.items()}
    mappings_big_reversed = {v: k for k, v in mappings_big.items()}

with open(node_embeddings_small_file) as nesf:
    node_embeddings_small = json.load(nesf)
    node_embeddings_small = {mappings_small_reversed[k]: v for k, v in node_embeddings_small.items()}

with open(node_embeddings_big_file) as nebf:
    node_embeddings_big = json.load(nebf)
    node_embeddings_big = {mappings_big_reversed[k]: v for k, v in node_embeddings_big.items()}

with open(gold_pairs_file) as gpf:
    gold_pairs = []
    for line in gpf:
        numbers_list = [int(num) for num in line.strip().split(";")]
        gold_pairs.append(numbers_list)

with open(exact_match_file) as file:
    exact_match = json.load(file)

In [19]:
gold_exact = list()
gold_not_exact = list()

for p in gold_pairs:
    if [p[0], p[1]] in exact_match:
        gold_exact.append([p[0], p[1]])
    else:
        gold_not_exact.append([p[0], p[1]])

node_embeddings_small_list = list()
node_ids_small_list = list()

node_embeddings_big_list = list()
node_ids_big_list = list()

for k, v in node_embeddings_small.items():
    node_ids_small_list.append(k)
    node_embeddings_small_list.append(v)

for k, v in node_embeddings_big.items():
    node_ids_big_list.append(k)
    node_embeddings_big_list.append(v)

In [20]:
tensor_small = torch.Tensor(node_embeddings_small_list)
tensor_big = torch.Tensor(node_embeddings_big_list)
node_order = util.semantic_search(tensor_small, tensor_big, top_k=top)

In [21]:
top_dict = dict()
for idx, (node_id, order) in enumerate(zip(node_ids_small_list, node_order)):
    items_list = list()
    for item in order:
        items_list.append((node_ids_big_list[item['corpus_id']], item['score']))
    top_dict[node_id] = items_list

# NEW

In [53]:
dogtags_path = "_input/dogtags/"

with open(dogtags_path + small + ".json") as df:
    dogtags_small = json.load(df)
    dogtags_small_dict = {key: value for key, value in dogtags_small.items()}

with open(dogtags_path + big + ".json") as df:
    dogtags_big = json.load(df)
    dogtags_big_dict = {key: value for key, value in dogtags_big.items()}

In [56]:
top_dict['24714']

[('23539', 0.8072959780693054),
 ('11827', 0.790683925151825),
 ('14196', 0.750228762626648),
 ('15847', 0.7447865009307861),
 ('50718', 0.7406191229820251),
 ('14793', 0.7362664341926575),
 ('76145', 0.7361975312232971),
 ('13378', 0.7265722751617432),
 ('41809', 0.7254675626754761),
 ('4541', 0.7253934741020203),
 ('25171', 0.7248602509498596),
 ('80785', 0.7237447500228882),
 ('1104', 0.723582923412323),
 ('1298', 0.7231607437133789),
 ('1754', 0.7231518030166626),
 ('2893', 0.7229169011116028),
 ('8540', 0.7221696376800537),
 ('10187', 0.7209693789482117),
 ('19025', 0.7204998135566711),
 ('8283', 0.7203250527381897),
 ('11452', 0.720008909702301),
 ('52538', 0.7196318507194519),
 ('33698', 0.7183557748794556),
 ('22662', 0.7181586623191833),
 ('52904', 0.716913640499115),
 ('12295', 0.7168094515800476),
 ('119044', 0.7160066366195679),
 ('24382', 0.7150982022285461),
 ('1355', 0.7149240374565125),
 ('92849', 0.7149120569229126),
 ('45338', 0.7141022682189941),
 ('1053', 0.71389091

In [59]:
mappings_small['24714']

'http://dbkwik.webdatacommons.org/stexpanded.wikia.com/resource/Magna_Roma'

In [57]:
mappings_big['23539']

'http://dbkwik.webdatacommons.org/memory-alpha.wikia.com/resource/892-IV_native'

In [58]:
mappings_big['11827']

'http://dbkwik.webdatacommons.org/memory-alpha.wikia.com/resource/892-IV'

In [49]:
str(dogtags_small[mappings_small['24714']])

'{\'abstract\': \'UFC 892-IV, called Magna Roma ("Great Rome") by its inhabitants, is a class M planet where evidence of Hodgkin\\\'s Law of Parallel Planetary Development was present in the 23rd century as a parallel of the Roman Empire existed there. (: "Bread and Circuses")In Orion Press continuity, the planet was called "Tellus" and System 892 had the Bayer classification of Beta Hydrae. It had been speculated that the planet\\\'s inhabitants had been transplanted from Earth by the Preservers 2,500 years before. (Orion Press: "The Return", TOS novel: Ex Machina)By the late 24th century, the Magna Romans had joined the Federation. (TNG novel: The Captains\\\' Honor)\', \'altLabel\': \'Magna Roma, 892-IV\', \'class\': \'Class_M_planet\', \'label\': \'Magna Roma\', \'planetName\': \'Magna Roma\', \'subject\': \'Category:Class_M_planets, Category:Alpha_Quadrant_planets\', \'surface\': \'Habitable\', \'system\': \'UFC-892\', \'type\': \'planet_info\'}'

In [52]:
str(dogtags_big[mappings_big[top_dict['24714'][0][0]]])

'{\'abstract\': \'The natives of the planet 892-IV were humanoids virtually indistinguishable from Human norms. The culture of 892-IV was considered a prime example of Hodgkin\\\'s Law, developing an analog of Earth\\\'s Roman Empire and Jesus Christ, about 2300 years after those events transpired on Earth.At the time of the USS Enterprise visit in 2268, the Roman Empire of 892-IV had yet to fall and a nascent Christianity was only beginning to flourish, in the early modern age of television.According to Spock, the natives spoke "colloquial English" of the mid-20th century. (: "")\', \'altLabel\': \'892-IV native\', \'imagecap\': \'Claudius_Marcus, Drusilla\', \'label\': \'892-IV native\', \'planet\': \'892\', \'subject\': \'Category:892-IV_natives, Category:Unnamed_species\', \'type\': \'Humanoid, sidebar_species\'}'

In [60]:
str(dogtags_big[mappings_big[top_dict['24714'][1][0]]])

'{\'abstract\': \'892-IV was the inhabited fourth planet of the 892 system. Classified as M-class, the planet was orbited by two moons and was the homeworld of the 892-IV natives, a pre-warp humanoid species.As Commander Spock, science officer of the USS Enterprise, noted, 892-IV had a striking resemblance to Earth. However, as much as there were similarities, there were differences. The proportion of land to water, density (5.5g), and atmosphere (78% nitrogen, 21% oxygen) were exactly like those on Earth. However, unlike Earth, the diameter at the equator (7,917 miles) and layout of the continents were different. (: "")\', \'altLabel\': \'Magna Roma, 892-IV, 892 IV\', \'class\': \'Class_M\', \'datestatus\': \'2267\', \'imageorbital\': \'892\', \'label\': \'892-IV\', \'location\': \'System_892\', \'name\': \'892\', \'nativeSpecies\': \'892\', \'orbitalcap\': \'892\', \'satellites\': \'2\', \'status\': \'Habitable\', \'subject\': \'Category:Homeworlds, Category:Planets, Category:Memory_

In [80]:
id_list = list()
str_list = list()
for i in range(0, 100):
    id_list.append(top_dict['24714'][i][0])
    str_list.append(
        [
            str(dogtags_small[mappings_small['24714']]),
            str(dogtags_big[mappings_big[top_dict['24714'][i][0]]])
        ]
    )

In [65]:
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('BAAI/bge-reranker-large')
model = AutoModelForSequenceClassification.from_pretrained('BAAI/bge-reranker-large')
model.eval()

pairs = str_list
with torch.no_grad():
    inputs = tokenizer(pairs, padding=True, truncation=True, return_tensors='pt', max_length=512)
    scores = model(**inputs, return_dict=True).logits.view(-1, ).float()
    print(scores)

tensor([ 2.6054,  3.4972,  0.8446,  0.0592, -3.4110, -1.0016, -2.7684, -1.7681,
        -0.8607, -1.3446, -1.1231, -2.6559, -1.5485, -1.8293,  0.0197, -2.1368,
        -2.4818, -2.0051, -2.2224, -2.6932, -1.7942, -1.6584, -2.0752,  0.0630,
        -2.1379, -1.0426, -3.0633, -2.1133, -1.5976, -0.3861,  0.0983, -3.4356,
        -1.2345, -0.6583,  0.3776, -1.7113, -2.6536, -3.2906, -2.5351, -0.9217,
        -1.6643, -0.9845, -0.8463, -2.1839, -2.2622, -2.9448, -3.0083, -1.8526,
        -2.8043, -1.6411, -0.2878, -1.5705, -1.0412, -1.1296, -1.7357, -1.8615,
        -1.3205, -2.0608, -2.7182, -3.1443, -2.8132, -3.5461, -3.7473,  0.0367,
        -1.6920, -2.0210, -1.9443, -2.2008, -3.4531, -5.4519, -2.7838, -3.2980,
        -1.4614, -2.1636, -0.5444, -0.2178, -3.2632, -2.4050, -2.2282, -1.8820,
        -0.9127, -2.4534, -2.7551, -1.9964, -2.0581, -1.7860, -2.2964, -3.1979,
        -1.9758, -2.4517, -1.2557, -1.9396, -2.0112, -2.5047, -2.6497, -2.0449,
        -4.5698, -2.5881, -3.8111, -3.36

In [86]:
max_index = torch.argmax(scores)
max_value = scores[max_index]

max_index_int = int(max_index.item())
max_value_float = float(max_value.item())

print(mappings_big[id_list[max_index_int]])

http://dbkwik.webdatacommons.org/memory-alpha.wikia.com/resource/892-IV


# END

In [23]:
print("############### SETTINGS ################")
print("From:            " + small)
print("To:              " + big)
print("Embeddings:      " + embeddings)

print("############ ALL GOLD PAIRS #############")
print("Count:           " + str(len(gold_pairs)))

found = 0
all_pairs = len(gold_pairs)
# for gold_pair in tqdm(gold_pairs):
for gold_pair in gold_pairs:
    for i in range(0, 1000):
        if top_dict.get(str(gold_pair[0]))[i][0] == str(gold_pair[1]):
            found += 1
            break
print("In Top 1000:     " + str(found) + " (" + f"{found / all_pairs * 100:.5f}" + "%)")

found = 0
all_pairs = len(gold_pairs)
# for gold_pair in tqdm(gold_pairs):
for gold_pair in gold_pairs:
    for i in range(0, 100):
        if top_dict.get(str(gold_pair[0]))[i][0] == str(gold_pair[1]):
            found += 1
            break
print("In Top 100:      " + str(found) + " (" + f"{found / all_pairs * 100:.5f}" + "%)")

found = 0
all_pairs = len(gold_pairs)
# for gold_pair in tqdm(gold_pairs):
for gold_pair in gold_pairs:
    for i in range(0, 10):
        if top_dict.get(str(gold_pair[0]))[i][0] == str(gold_pair[1]):
            found += 1
            break
print("In Top 10:       " + str(found) + " (" + f"{found / all_pairs * 100:.5f}" + "%)")

found = 0
all_pairs = len(gold_pairs)
# for gold_pair in tqdm(gold_pairs):
for gold_pair in gold_pairs:
    if top_dict.get(str(gold_pair[0]))[0][0] == str(gold_pair[1]):
        found += 1
print("Top 1:           " + str(found) + " (" + f"{found / all_pairs * 100:.5f}" + "%)")

print("############## EXACT MATCH ##############")
print("Count:           " + str(len(gold_exact)))

found = 0
all_pairs = len(gold_exact)
# for gold_pair in tqdm(gold_exact):
for gold_pair in gold_exact:
    for i in range(0, 1000):
        if top_dict.get(str(gold_pair[0]))[i][0] == str(gold_pair[1]):
            found += 1
            break
print("In Top 1000:     " + str(found) + " (" + f"{found / all_pairs * 100:.5f}" + "%)")

found = 0
all_pairs = len(gold_exact)
# for gold_pair in tqdm(gold_exact):
for gold_pair in gold_exact:
    for i in range(0, 100):
        if top_dict.get(str(gold_pair[0]))[i][0] == str(gold_pair[1]):
            found += 1
            break
print("In Top 100:      " + str(found) + " (" + f"{found / all_pairs * 100:.5f}" + "%)")

found = 0
all_pairs = len(gold_exact)
# for gold_pair in tqdm(gold_exact):
for gold_pair in gold_exact:
    for i in range(0, 10):
        if top_dict.get(str(gold_pair[0]))[i][0] == str(gold_pair[1]):
            found += 1
            break
print("In Top 10:       " + str(found) + " (" + f"{found / all_pairs * 100:.5f}" + "%)")

found = 0
all_pairs = len(gold_exact)
# for gold_pair in tqdm(gold_exact):
for gold_pair in gold_exact:
    if top_dict.get(str(gold_pair[0]))[0][0] == str(gold_pair[1]):
        found += 1
print("Top 1:           " + str(found) + " (" + f"{found / all_pairs * 100:.5f}" + "%)")

print("############ NOT EXACT MATCH ############")
print("Count:           " + str(len(gold_not_exact)))

found = 0
all_pairs = len(gold_not_exact)
# for gold_pair in tqdm(gold_not_exact):
for gold_pair in gold_not_exact:
    for i in range(0, 1000):
        if top_dict.get(str(gold_pair[0]))[i][0] == str(gold_pair[1]):
            found += 1
            break
print("In Top 1000:     " + str(found) + " (" + f"{found / all_pairs * 100:.5f}" + "%)")

found = 0
all_pairs = len(gold_not_exact)
# for gold_pair in tqdm(gold_not_exact):
for gold_pair in gold_not_exact:
    for i in range(0, 100):
        if top_dict.get(str(gold_pair[0]))[i][0] == str(gold_pair[1]):
            found += 1
            break
print("In Top 100:      " + str(found) + " (" + f"{found / all_pairs * 100:.5f}" + "%)")

found = 0
all_pairs = len(gold_not_exact)
# for gold_pair in tqdm(gold_not_exact):
for gold_pair in gold_not_exact:
    for i in range(0, 10):
        if top_dict.get(str(gold_pair[0]))[i][0] == str(gold_pair[1]):
            found += 1
            break
print("In Top 10:       " + str(found) + " (" + f"{found / all_pairs * 100:.5f}" + "%)")

found = 0
all_pairs = len(gold_not_exact)
# for gold_pair in tqdm(gold_not_exact):
for gold_pair in gold_not_exact:
    if top_dict.get(str(gold_pair[0]))[0][0] == str(gold_pair[1]):
        found += 1
print("Top 1:           " + str(found) + " (" + f"{found / all_pairs * 100:.5f}" + "%)")

############### SETTINGS ################
From:            stexpanded
To:              memoryalpha
Embeddings:      dogtag_bgelarge
############ ALL GOLD PAIRS #############
Count:           1779
In Top 1000:     1766 (99.26925%)
In Top 100:      1732 (97.35807%)
In Top 10:       1672 (93.98539%)
Top 1:           1474 (82.85554%)
############## EXACT MATCH ##############
Count:           1620
In Top 1000:     1611 (99.44444%)
In Top 100:      1584 (97.77778%)
In Top 10:       1537 (94.87654%)
Top 1:           1371 (84.62963%)
############ NOT EXACT MATCH ############
Count:           159
In Top 1000:     155 (97.48428%)
In Top 100:      148 (93.08176%)
In Top 10:       135 (84.90566%)
Top 1:           103 (64.77987%)
